In [18]:
# 가장 뜨는 제주도 핫플레이스는? - 크롤링을 통한 분석
# 인스타그램에 접속한 후 키워드 검색 "제주도맛집"

In [19]:
# https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91(제주도맛집)/
# https://www.instagram.com/explore/tags/%EB%A7%88%EC%84%9D%EB%A7%9B%EC%A7%91(마석맛집)/
# 인스타그램의 태그 구성을 보면 태그의 제목이 가장 끝쪽에 위치함
# www.instagram.com/explore/tags/{키워드} 구성으로 url 구성가능


In [20]:
# 인스타그램 검색 결과 URL 만들어서 접속하기
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [21]:
from selenium import webdriver

driver = webdriver.Chrome(r'C:\Users\82104\Desktop\프로그래밍\chromedriver.exe') 
# 주소의 표현방법은 1. 상대참조 2. 절대참조 방법이 있는데, 절대참조는 r'~\~\~' 을 넣어서 주소를 참조하는 방법과 \\~~\\~~\\ 방법의 참조가 있음

word = '제주도맛집'
url = insta_searching(word)
driver.get(url)

# 주소에 안 넣었었음 chromedriver.exe
# tags -> tags/ 로 수정

In [22]:
# 첫번째 게시글 보기
import time # 시간들이기

def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0") 
    first.click() # 태그명이 div이며, class명이 _9AhH0인 요소를 찾아 클릭
    time.sleep(3) # 3초간 대기

In [23]:
# 게시글 정보보기 - #해시태그 정보 추출
# 게시글의 정보를 가져왔으면, 다른 게시글로 이동하고 게시글 정보를 수집하는 과정 반복 수행
import re # 정규표현식에 쓰기 위한 라이브러리

def get_content(driver):
    # 1. 현재 페이지의 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')

    # 2. 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
        # 태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택
        # 여러 태그 중 첫 번째 태그 선택, 해당 태그의 텍스트 부분
    except:
        content = '' # 오류는 공백으로 반환
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식)
    tags = re.findall(r'#[^\s#,\\]+', content) 
    # 해시태그를 본문에서도 쓸 수가 있기 때문에 글을 다 쓰고 난 글을 가져와야 함

    # 4. 작성일자 정보 가져오기
    try:
        date = soup.select('time._1o9PC.Nzb55') [0]['datetime'][:10]
    # 태그명이 time, class명이 _1o9PC, Nzb55인 태그 모두 선택 -오류수정-> element값을 복사해오는게 정확함
    # 해당 태그 첫 번째 선택, 해당 태그의 datetime 속성값, 앞에서 부터 10자리 추출
    except IndexError:
        date = ''
    # indexerror가 날 시에는 값 추출 x

    # 5. 좋아요 수 가져오기 (동영상 자료구조는 어떻게 불러와야 하는지?)
    try:
        like = soup.select('div.Nm9Fw > button')[0].text [4:-1]
        # 조회수가 뜨는 동영상 자료구조는 만들기 어렵다
        # 태그명이 div, class명이 Nm9Fw인 태그 바로 아래에 있는 button 태그를 모두 선택
        # 여러 태그 중 첫 번째 태그 선택, 해당 테크의 텍스트 선택 후 5번째 글자부터 오른쪽 끝에서 두번째 글자 선택
    except:
        like = 0
        # 좋아요가 없으면 0으로 산출
        # 동영상 구조는?
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.M30cS')[0].text
        # 태그명이 div, class명이 M30cS인 태그의 첫번째 텍스트 부분 추출
    except:
        place = ''
    
    # 7. 수집한 정보 저장
    data = [content,date,like,place,tags]
    return data


In [24]:
# 다음 게시글 열기 - 해당 게시글 정보 수집 완료 후, 다음 게시글로 이동하기
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)


In [25]:
# 1. 크롬 브라우저 열기
# 2. 인스타 검색페이지 url 만들기
# 3. 검색페이지 접속하기 ex) 제주도맛집
# 4. 첫 번재 게시물 클릭하기
# 5. 비어있는 변수 만들기
# 6. 게시글 정보 가져오기
# 7. 게시글 정보 변수에 추가한 후 다음 게시글 선택하기
# 8. 게시글 정보 수집 완료

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬 브라우저 열기
driver = webdriver.Chrome(r'C:\Users\82104\Desktop\프로그래밍\chromedriver.exe')

# 1-1 인스타그램에 접속할 때 로그인을 해야 들어갈 수 있는 환경으로 변경
#driver.get('https://www.instargram.com')
#time.sleep(2)

# 2. 인스타그램 검색페이지 url 만들기
word = '제주도맛집'
url = insta_searching(word) # 함수 insta_searching()

# 3. 검색 페이지 접속하기
driver.get(url)
time.sleep(3)

# 3-1 인스타 로그인 하기
email = 'eb3434@naver.com'   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'yoon0924!' ### 비번 정보 수정 필요
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

# 3-2 로그인 정보를 저장하시겠어요?
login = driver.find_element_by_css_selector('div.cmbtv')
login.click()
time.sleep(3)

# 3-3 알림 설정 - 나중에 하기
later = driver.find_element_by_css_selector('div.mt3GC')
later.click()
time.sleep(3)

# 3-4 검색페이지 재접속하기
driver.get(url)
time.sleep(3)

# 4. 첫 번째 게시글 열기
select_first(driver)

# 5. 비어있는 변수 만들기
results = []

# 6~8. 여러 게시물 수집하기
target = 10  # 크롤링할 게시글 수
for i in range(target):
    data = get_content(driver) # 게시글 정보 가져오기
    results.append(data)
    move_next(driver)

print(results[:2])

# ---> 22     date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
# IndexError: list index out of range
# try/except 구문 넣고 다시 시작 
# 크롤링할 게시글 수를 올리면 문제가 생기는데, 뭐가 문제지..?
# 아무래도 웹페이지상 생기는 오류같다. indexerror가 날 때 생기는 오류는 date = 0 값으로 처리했다
# SessionNotCreatedException: Message: session not created from chrome not reachable 이 오류는 실행된 크롬 창을 닫았을 때 생기게 됨

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.cmbtv"}
  (Session info: chrome=87.0.4280.141)


In [26]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel(r'C:\Users\82104\Desktop\프로그래밍\파이썬 자료\제주도 핫플레이스\crawling_Jeju.xlsx')

# ValueError: No engine for filetype: ''
# SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape
# xlsx 파일 형태를 넣지 않았기 때문에 오류 발생. 

NameError: name 'results' is not defined

In [27]:
# 엑셀에 있는 중복된 내용을 제거 후 통합 저장하기
import pandas as pd
jeju_insta_df = pd.DataFrame( [ ] )

folder = 'C:\\Users\\82104\\Desktop\\프로그래밍\\파이썬 자료\\datasalon-master\\01_초판\\4_Jeju_Hotplace\\files\\'
f_list = ['3_1_crawling_jejudoGwanGwang.xlsx','3_1_crawling_jejudoMatJip.xlsx','3_1_crawling_jejuMatJip.xlsx','3_1_crawling_jejuYeoHang.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df.append(temp)

jeju_insta_df.columns = ['content','data','like','place','tags']

#SyntaxError: EOL while scanning string literal -> files\ 에서 files로 변경
#FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\82104\\Desktop\\프로그래밍\\파이썬 자료\\datasalon-master\\02_개정판\\5_Jeju_Hotplace\\files1_crawling_jejudoGwanGwang.xlsx'
#ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements